<a href="https://colab.research.google.com/github/Isotr0py/Acolite-GEE-Colab/blob/main/Acolite_Notebook/Acolite-GEE-Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Initialize Runtime
#@markdown Mount Google Drive
Mount_Drive=True #@param {type:"boolean"}
if Mount_Drive:
  from google.colab import drive
  drive.mount('/content/gdrive')
# acolite requirements
!pip install netcdf4

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.7 MB/s eta 0:00:00


In [ ]:
# @title Authenticate and Initialize GEE
import ee
import geemap

#@markdown Your GEE project to initialize
project = "ee-acolite" # @param {type:"string"}
ee.Authenticate()
ee.Initialize(project=project)
# ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com',project='ee-entr0py')
Map=geemap.Map()

In [ ]:
# @title Clone acolite repository
# @markdown Download [Acolite](https://github.com/acolite/acolite) from Github
# Acolite can't be installed as library. It's hard coded.
%cd /content/
Release = "20231023.0" #@param ["20231023.0"]
# @markdown **<font color=red>Warning: use_latest_repo will download `main` branch instead of Release version (unstable)!!**
use_latest_repo = False #@param{type:"boolean"}
if use_latest_repo:
  !git clone --depth 1 https://github.com/acolite/acolite
else:
  !git clone --depth 1 https://github.com/acolite/acolite --branch $Release
%cd acolite
# !git clone https://github.com/acolite/acolite_luts data/LUT

/content
Cloning into 'acolite'...
remote: Enumerating objects: 849, done.
remote: Counting objects: 100% (849/849), done.
remote: Compressing objects: 100% (762/762), done.
remote: Total 849 (delta 101), reused 626 (delta 66), pack-reused 0
Receiving objects: 100% (849/849), 3.82 MiB | 19.35 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Note: switching to '81dd9d34224c535166de3c1fab9c2a057ef8b615'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

/content/acolite


In [ ]:
import ee
import geemap

#@title Setting ROI and Run acolite
# @markdown **<font color=cyan>:)  Readme**

# @markdown - **<font color=cyan>Setting the parameters**

# @markdown - **<font color=cyan>First Run will display a GEE map**

# @markdown - **<font color=cyan>Draw your ROI on map**

# @markdown - **<font color=cyan>Re-run the cell to start the process!**



# @markdown Basic settings
isodate_start='2021-12-05' #@param {type:'date'}
isodate_end='2021-12-06' #@param {type:'date'}
sensors = "S2_MSI" # @param ["S2_MSI", "L9_OLI", "L8_OLI", "L7_ETM", "L5_TM"]
output_dir = "/content/example" # @param {type:"string"}

run_hybrid_dsf = False #@param {type:"boolean"}
run_offline_dsf = True #@param {type:"boolean"}


settings = {}
gee_settings = {}

gee_settings["output"] = output_dir
if sensors == "S2_MSI":
  gee_settings["sensors"] = "S2A_MSI, S2B_MSI"
else:
  gee_settings["sensors"] = sensors
gee_settings["isodate_start"] = isodate_start
gee_settings["isodate_end"] = isodate_end
gee_settings["strict_subset"] = True
gee_settings["run_hybrid_dsf"] = run_hybrid_dsf
gee_settings["run_offline_dsf"] = run_offline_dsf

settings["merge_tiles"] = True
settings["merge_zones"] = True
settings["map_l2w"] = True

#@markdown L2W parameters
settings["l2w_parameters"]= []
Rrs = False #@param {type:"boolean"}
if Rrs:
  settings["l2w_parameters"].append('Rrs_*')

rhorc = False #@param {type:"boolean"}
if rhorc:
  settings["l2w_parameters"].append('rhorc_*')

custom_l2w_parameters = "spm_nechad2016"  #@param {type:"string"}
if custom_l2w_parameters:
  settings["l2w_parameters"] += custom_l2w_parameters.split(",")

# GEEMAP
image_collection = {
    "S2_MSI": "COPERNICUS/S2_HARMONIZED",
    "L9_OLI": "LANDSAT/LC09/C02/T1_TOA",
    "L8_OLI": "LANDSAT/LC08/C02/T1_TOA",
    "L7_ETM": "LANDSAT/LE07/C02/T1_TOA",
}
image = ee.ImageCollection(image_collection[sensors]).filterDate(isodate_start,isodate_end);

#Visuaization

visualization = {
    "sentinel": {
      'bands': ['B4','B3','B2'],
      'min': 0,
      'max': 2500,
    },
    "landsat": {
      'bands': ['B4','B3','B2'],
      'min': 0,
      'max': 0.2,
    },
}
if sensors == "S2_MSI":
  vis = visualization["sentinel"]
else:
  vis = visualization["landsat"]

Map.addLayer(image, vis, name=sensors);
display(Map)
if Map.draw_features:
  feature = ee.FeatureCollection(Map.draw_features)
  coordinates = feature.geometry().bounds().coordinates().getInfo()[0]
  lon = [x[0] for x in coordinates]
  lat = [x[1] for x in coordinates]
  S,W,N,E = min(lat),min(lon),max(lat),max(lon)
  gee_settings["limit"] = [S,W,N,E]

  import acolite.gee
  acolite.gee.agh_run(settings=gee_settings, acolite_settings=settings)


In [ ]:
!rm -rf /content/example